# Criteo Sponsored Search Conversion Log Dataset Analysis

## Resources
Dataset: [Criteo Sponsored Search Conversion Log Dataset](https://ailab.criteo.com/criteo-sponsored-search-conversion-log-dataset/)

Paper: [Reacting to Variations in Product Demand: An Application for
Conversion Rate (CR) Prediction in Sponsored Search
](https://arxiv.org/pdf/1806.08211.pdf)

## Shortform Overview of the data
- Entire dataset spans 90 days of logs
- Attribution Window used is 30 days
- Each row in the dataset represents an action (i.e. click) performed by the user on a product related advertisement.
- Each row may or may not have a conversion with it. If Sale = 1, then there was a conversion.
- The data has been sub-sampled, and it's not clear how the sampling was done. So, it could be that we have impartial user stories. For instance, multiple impressions might have _actually_ led to a conversion, but that conversion might be missing from the dataset. Similarly, multiple impressions might have _actually_ been recorded, but some impressions might be missing from the dataset. This has implications for accurately recording privacy budget deductions.

## Overview of the data
(Copied from the data source website. Emphasis is mine)

This dataset contains logs obtained from Criteo Predictive Search (CPS). CPS, offers an automated end-to-end solution using sophisticated machine learning techniques to improve Google Shopping experience using robust, predictive optimization across every aspect of the advertiser’s campaign. CPS in general has two main aims : (1) Retarget high-value users via behavioral targeting such that the bids are based on each user’s likelihood to make a purchase. (2) Increase ROI using a bidding strategy which incorporates the effects of product characteristics, user intent, device and user behavior.

**Each row in the dataset represents an action (i.e. click) performed by the user on a product related advertisement**. The product advertisement was shown to the user, post the user expressing an intent via an online search engine.  Each row in the dataset, contains information about the product characteristics (age, brand, gender, price), time of the click ( subject to uniform shift), user characteristics and device information. The **logs also contain information on whether the clicks eventually led to a conversion (product was bought) within a 30 day window and the time between click and the conversion**.

**This dataset represents a sample of 90 days of Criteo live traffic data**. Each line corresponds to one click (product related advertisement) that was displayed to a user. For each advertisement, we have detailed information about the product. Further, we also provide information on whether the click led to a conversion, amount of conversion and the time between the click and the conversion. **Data has been sub-sampled** and anonymized so as not to disclose proprietary elements.

In [191]:
# import pandas as pd
import modin.pandas as pd
import numpy as np
from datetime import datetime
import os
import plotly.express as px

os.environ["MODIN_ENGINE"] = "ray"

### Dataset Retrieval
This notebook assumes you already have the dataset downloaded and in the current directory. If you do not, uncomment and run the following cell. Note that the uncompressed data is ~6 GB.

In [2]:
# !wget http://go.criteo.net/criteo-research-search-conversion.tar.gz
# !tar -xzf criteo-research-search-conversion.tar.gz

In [192]:
DATA_FILE = 'Criteo_Conversion_Search/CriteoSearchData'
dtype={
    "Sale": np.int32,
    "SalesAmountInEuro": np.float64,
    "Time_delay_for_conversion": np.int32,
    "click_timestamp": np.int32,
    "nb_clicks_1week": pd.Int64Dtype(),
    "product_price": np.float64,
    "product_age_group": str,
    "device_type": str,
    "audience_id": str,
    "product_gender": str,
    "product_brand": str,
    "product_category1": str,
    "product_category2": str,
    "product_category3": str,
    "product_category4": str,
    "product_category5": str,
    "product_category6": str,
    "product_category7": str,
    "product_country": str,
    "product_id": str,
    "product_title": str,
    "partner_id": str,
    "user_id": str,
}
na_values={
    "click_timestamp": "0",
    "nb_clicks_1week": "-1",
    "product_price": "-1",
    "product_age_group": "-1",
    "device_type": "-1",
    "audience_id": "-1",
    "product_gender": "-1",
    "product_brand": "-1",
    "product_category1": "-1",
    "product_category2": "-1",
    "product_category3": "-1",
    "product_category4": "-1",
    "product_category5": "-1",
    "product_category6": "-1",
    "product_category7": "-1",
    "product_country": "-1",
    "product_id": "-1",
    "product_title": "-1",
    "partner_id": "-1",
    "user_id": "-1",
}
columns_to_drop = [
    'product_category1', 'product_category2', 'product_category3', 'product_category4',
    'product_category5', 'product_category6', 'product_category7', 'nb_clicks_1week', 'device_type',
    'product_title', 'product_brand', 'product_gender', 'audience_id', 'product_age_group', "product_country"
]

In [194]:
df = pd.read_csv(DATA_FILE, names=dtype.keys(), dtype=dtype, na_values=na_values, header=None, sep="\t")
df = df.drop(columns=columns_to_drop)
df = df.dropna(subset=['partner_id', 'user_id', "product_id"])
df.head()

,Sale,SalesAmountInEuro,Time_delay_for_conversion,click_timestamp,product_price,product_id,partner_id,user_id
0,0,-1.0,-1,1598891820,0.00,A66DB02AC1726A8D79C518B7F7AB79F0,E3DDEB04F8AFF944B11943BB57D2F620,493CFB4A87C50804C94C0CF76ABD19CD
1,0,-1.0,-1,1598925284,0.00,4C6C62203B4CE5AA6DFCF17F2604DC37,BD01BAFAE73CF38C403978BBB458300C,D0EBCD4402172AE3AA6FD21FB77BDE84
4,0,-1.0,-1,1598905245,0.00,5F4AFF2693601C2EFFE9AF2C1ED4222E,E3DDEB04F8AFF944B11943BB57D2F620,703B8CFC8D65A67BDF96595CCF992D27
5,1,89.9,442485,1598929598,49.95,C806140F3AF11759BEF4E3B79B51588F,743B1EE3A39E06D855A72B3B66D501D0,E94BA61ACE2488B7DA7920736B82DFBD
6,0,-1.0,-1,1598940008,0.00,EE31CDA023E1237FBAD9C75E08C1D2BF,E3DDEB04F8AFF944B11943BB57D2F620,B361216FD8F17C7F748DFAC7EEA9C231


In [236]:
df["click_datetime"] = df["click_timestamp"].apply(lambda x: datetime.fromtimestamp(x))
df["click_day"] = df["click_datetime"].apply(
    lambda x: (7 * (x.isocalendar().week - 1)) + x.isocalendar().weekday
)
min_click_day = df["click_day"].min()
df["click_day"] -= min_click_day

df["conversion_timestamp"] = df["Time_delay_for_conversion"] + df["click_timestamp"]
df["conversion_datetime"] = df["conversion_timestamp"].apply(
    lambda x: datetime.fromtimestamp(x)
)
df["conversion_day"] = df["conversion_datetime"].apply(
    lambda x: (7 * (x.isocalendar().week - 1)) + x.isocalendar().weekday
)
df["conversion_day"] -= min_click_day

impressions = df[["click_timestamp", "click_day", "user_id", "partner_id", "product_id"]]
conversions = pd.DataFrame(df.loc[df.Sale == 1])[
    [
        "conversion_timestamp",
        "conversion_day",
        "user_id",
        "partner_id",
        "product_id",
        # "SalesAmountInEuro",
    ]
]
# conversions["SalesAmountInEuro"] = conversions["SalesAmountInEuro"].round(decimals=0)


In [196]:
conversions

,conversion_timestamp,conversion_day,user_id,partner_id,product_id,SalesAmountInEuro
5,1599372083,34,E94BA61ACE2488B7DA7920736B82DFBD,743B1EE3A39E06D855A72B3B66D501D0,C806140F3AF11759BEF4E3B79B51588F,90.0
7,1598908419,28,A00E7A8538C058366DA15181CC56FD99,E3DDEB04F8AFF944B11943BB57D2F620,9E12223B51B3D14342F2DBD05EA8B302,158.0
14,1598974575,29,2324579BDAC61ED6E3BBBB799516AA6A,E3DDEB04F8AFF944B11943BB57D2F620,5CA21A95401BDEEB84C86C0470ADB770,38.0
30,1599416894,34,1967F0A0EF8409D375C095E63B2AF8A4,E3DDEB04F8AFF944B11943BB57D2F620,D362EA2039B5429CC545CB390E278270,26.0
37,1598914918,28,1456A8090909E75AD8E170CE27520A47,E3DDEB04F8AFF944B11943BB57D2F620,E09935C131B17B9A7324CC7116183F63,46.0
...,...,...,...,...,...,...
15995566,1599157854,31,77330315AD617432B05B34A13F930899,CC14152DD0806823ED88BA55D9A5240B,EAA97A165A02E3CD92E85AAF16D744F3,503.0
15995573,1598022328,18,46C68692A885E300478FDF3814FEC9F2,4691EA5C1795A32C20FCEF274CC28BE5,9A57F2B95F8B2B2607B86381889BD533,219.0
15995583,1597006010,6,E827FB2288AADB0B77A61AB5FE55ABB8,CE13C4ECD6D0EF4A9F02EB0E17B45766,65DCAF5BAF7EA0EC8DD54C3C4CC77AAB,82.0
15995617,1597671703,14,4EAE026A6A5113100A8B769138C5BC52,CC14152DD0806823ED88BA55D9A5240B,6BC44EBC7707DC772C286589AD885957,75.0


In [217]:
adsc = conversions.query("partner_id=='9D9E93D1D461D7BAE47FB67EC0E01B62' or partner_id=='F122B91F6D102E4630817566839A4F1F' or partner_id=='9FF550C0B17A3C493378CB6E2DEEE6E4'")
adsc.groupby("partner_id").size()

partner_id
9D9E93D1D461D7BAE47FB67EC0E01B62    55532
9FF550C0B17A3C493378CB6E2DEEE6E4    46706
F122B91F6D102E4630817566839A4F1F    50102
dtype: int64

In [231]:
def hash_to_buckets(s):
    hash_value = hash(s)
    normalized_hash = (hash_value % 10000) / 10000
    if normalized_hash < 1/3:
        return 0
    elif normalized_hash < 2/3:
        return 1
    else:
        return 2

def get_epsilon_from_accuracy(n):
    s=1 
    a=0.05
    b=0.01

    epsilon = s * math.log(1/b) / (n * a)
    return epsilon
    
adsc["product_id_group"] = adsc["product_id"].apply(hash_to_buckets)
x = adsc.groupby(["partner_id", "product_id_group"]).size().reset_index(name="count")
x["epsilon"] = x["count"].apply(get_epsilon_from_accuracy)
adsc = adsc.merge(x, on=["partner_id", "product_id_group"], how="left")
# x

,partner_id,product_id_group,count,epsilon
0,9D9E93D1D461D7BAE47FB67EC0E01B62,0,18591,0.004954
1,9D9E93D1D461D7BAE47FB67EC0E01B62,1,18578,0.004958
2,9D9E93D1D461D7BAE47FB67EC0E01B62,2,18363,0.005016
3,9FF550C0B17A3C493378CB6E2DEEE6E4,0,15271,0.006031
4,9FF550C0B17A3C493378CB6E2DEEE6E4,1,15728,0.005856
5,9FF550C0B17A3C493378CB6E2DEEE6E4,2,15707,0.005864
6,F122B91F6D102E4630817566839A4F1F,0,16865,0.005461
7,F122B91F6D102E4630817566839A4F1F,1,16868,0.005460
8,F122B91F6D102E4630817566839A4F1F,2,16369,0.005627


I have three advertiser. Now let's define 3 queries for them.

1) Count across all conversions on product-id-group=0
2) Count across all conversions on product-id-group=1
3) Count across all conversions on product-id-group=2

In [235]:
adsc = adsc.merge(x, on=["partner_id", "product_id_group"], how="left")
adsc

,conversion_timestamp,conversion_day,user_id,partner_id,product_id,SalesAmountInEuro,product_id_group,count,epsilon
0,1598927451,28,2A99B0C23A6292B2F9D67026475C498B,F122B91F6D102E4630817566839A4F1F,607BB16E7655E60B22BCF0A2891FFD15,17.0,0,16865,0.005461
1,1598930857,28,4881DEE89F3B163552969251373967F2,9D9E93D1D461D7BAE47FB67EC0E01B62,C13B76030349E3DA3873FDE961AB218A,25.0,0,18591,0.004954
2,1599562787,36,4B40B3E90E6BDFEC6C67F2AB576CBF21,9D9E93D1D461D7BAE47FB67EC0E01B62,926797A01262660C0EF79050564AA125,29.0,1,18578,0.004958
3,1598928484,28,E2AA2583AFF52CE99B05C6BA9C76233A,F122B91F6D102E4630817566839A4F1F,8F8B3B076D37867EE184AC414EF1B48C,47.0,2,16369,0.005627
4,1598907404,28,AEEE173C54AE70F1534BD591B4A9455A,9D9E93D1D461D7BAE47FB67EC0E01B62,57BBDC31EB969B45990B43A7151B429E,105.0,0,18591,0.004954
...,...,...,...,...,...,...,...,...,...
152335,1598129686,19,538003E6B9540952BA8BCD17D0AF52BD,F122B91F6D102E4630817566839A4F1F,62B3801904C56B49B3F1A97CE8E8499A,15.0,1,16868,0.005460
152336,1598646418,25,35CC6594B46F229AD306F03BA58CA125,9D9E93D1D461D7BAE47FB67EC0E01B62,88E54D4B22D6ADFB015A1A97E2230442,283.0,0,18591,0.004954
152337,1598129107,19,7DCB5AFBE1E67D13B8B4DCA1F67908C5,F122B91F6D102E4630817566839A4F1F,C2CE7B42D6D1422C07BE6D22B6F90EDF,101.0,1,16868,0.005460
152338,1598638113,25,5DC3C1BE030380D6DF092B664AC4164B,9D9E93D1D461D7BAE47FB67EC0E01B62,9430FE00409DEF2C47C76A1E0C2E00B3,125.0,2,18363,0.005016


In [225]:
a = adsc.groupby("partner_id")["conversion_day"].median()
a

partner_id
9D9E93D1D461D7BAE47FB67EC0E01B62    53.0
9FF550C0B17A3C493378CB6E2DEEE6E4    61.0
F122B91F6D102E4630817566839A4F1F    50.0
Name: conversion_day, dtype: float64

In [201]:
# adsc.query("partner_id == '9D9E93D1D461D7BAE47FB67EC0E01B62'")["SalesAmountInEuro"].describe()
# a = len(adsc.query("partner_id == '9D9E93D1D461D7BAE47FB67EC0E01B62' and SalesAmountInEuro > 350"))
# a

In [202]:
# adsc.query("partner_id == 'F122B91F6D102E4630817566839A4F1F'")["SalesAmountInEuro"].describe()
# a = len(adsc.query("partner_id == 'F122B91F6D102E4630817566839A4F1F' and SalesAmountInEuro > 170"))
# a

In [203]:
# adsc.query("partner_id == '9FF550C0B17A3C493378CB6E2DEEE6E4'")["SalesAmountInEuro"].describe()
# a = len(adsc.query("partner_id == '9FF550C0B17A3C493378CB6E2DEEE6E4' and SalesAmountInEuro > 350"))
# a

In [204]:
# adsc.query("partner_id == '9D9E93D1D461D7BAE47FB67EC0E01B62' and product_id_group==0")["SalesAmountInEuro"].describe()
# a = len(adsc.query("partner_id == '9D9E93D1D461D7BAE47FB67EC0E01B62' and product_id_group==0 and SalesAmountInEuro > 350"))
# a

In [205]:
# adsc.query("partner_id == 'F122B91F6D102E4630817566839A4F1F' and product_id_group==0")["SalesAmountInEuro"].describe()
# a = len(adsc.query("partner_id == 'F122B91F6D102E4630817566839A4F1F' and product_id_group==0 and SalesAmountInEuro > 170"))
# a

In [206]:
# adsc.query("partner_id == '9FF550C0B17A3C493378CB6E2DEEE6E4' and product_id_group==0")["SalesAmountInEuro"].describe()
# a = len(adsc.query("partner_id == '9FF550C0B17A3C493378CB6E2DEEE6E4' and product_id_group==0 and SalesAmountInEuro > 350"))
# a

In [207]:
# adsc.query("partner_id == '9D9E93D1D461D7BAE47FB67EC0E01B62' and product_id_group==1")["SalesAmountInEuro"].describe()
# a = len(adsc.query("partner_id == '9D9E93D1D461D7BAE47FB67EC0E01B62' and product_id_group==1 and SalesAmountInEuro > 350"))
# a

In [208]:
# adsc.query("partner_id == 'F122B91F6D102E4630817566839A4F1F' and product_id_group==0")["SalesAmountInEuro"].describe()
# a = len(adsc.query("partner_id == 'F122B91F6D102E4630817566839A4F1F' and product_id_group==0 and SalesAmountInEuro > 170"))
# a

In [209]:
# adsc.query("partner_id == '9FF550C0B17A3C493378CB6E2DEEE6E4'")["SalesAmountInEuro"].describe()
# a = len(adsc.query("partner_id == '9FF550C0B17A3C493378CB6E2DEEE6E4' and SalesAmountInEuro > 350"))
# a

In [ ]:
total_impressions = impressions.shape[0]
total_conversions = conversions.shape[0]
unique_user_count = len(df.groupby(['user_id']).count())
unique_partner_count = len(df.groupby(['partner_id']).count())
unique_product_count = len(df.groupby(['product_id']).count())
unique_partner_product_count = len(df.groupby(['partner_id', 'product_id']).size())

print("total impressions:", total_impressions, "total conversions:", total_conversions)
print("conversion rate:", total_conversions/total_impressions*100, "%")
print("unique users:", unique_user_count)
print("unique partners:", unique_partner_count)
print("unique products:", unique_product_count)
print("unique per partner products :", unique_partner_product_count)

In [ ]:
iuser_counts = impressions.groupby(['user_id']).size().reset_index(name="count")
iuser_counts.describe()

In [ ]:
cuser_counts = conversions.groupby(['user_id']).size().reset_index(name="count")
cuser_counts.describe()

In [ ]:
iday_counts = impressions.groupby(['click_day']).size().reset_index(name="count")
iday_counts.describe()

In [ ]:
iuser_day_counts = impressions.groupby(['user_id', 'click_day']).size().reset_index(name="count")
iuser_day_counts.describe()

In [ ]:
cday_counts = conversions.groupby(['conversion_day']).size().reset_index(name="count")
cday_counts.describe()

In [ ]:
cday_counts = conversions.groupby(['conversion_day']).size().reset_index(name="count")
cday_counts.describe()

In [ ]:
cuser_day_counts = conversions.groupby(['user_id', 'conversion_day']).size().reset_index(name="count")
cuser_day_counts.describe()

In [ ]:
# print(user_day_counts.loc[user_day_counts['count'] == 1].shape)
# print(user_day_counts.loc[user_day_counts['count'] > 1].shape)
# print(user_day_counts.loc[user_day_counts['count'] > 2].shape)
# print(user_day_counts.loc[user_day_counts['count'] > 3].shape)
# print(user_day_counts.loc[user_day_counts['count'] > 4].shape)
# print(user_day_counts.loc[user_day_counts['count'] > 5].shape)


In [ ]:
# user_counts = user_counts.sort_values(["count"], ascending=False,)
# user_counts

In [ ]:
# iuser = impressions.query("user_id == 'C8C869CD45415BA13541D602D8EA277E'")
# cuser = conversions.query("user_id == 'C8C869CD45415BA13541D602D8EA277E'")
# iuser
# cuser

In [ ]:
conversion_user_day_counts = conversions.groupby(['user_id', 'conversion_day']).size().reset_index(name="count")
conversion_user_day_counts.describe()

In [ ]:
# fig = px.ecdf(user_day_counts, x="count")
# fig.show()

In [ ]:
# df.query("partner_id=='319A2412BDB0EF669733053640B80112' and product_id=='C9A3F830655829E5E924423E7417AAB4'")